# 6.5 Lab: Linear Models and Regularization Methods

## 6.5.1 Subset Selection Methods

In [ ]:
"""
This notebook contains the code for best subset selection, 
so this notebook may take longer time to run, for faster run, make
max_feature into a smaller number
"""
max_feature = 3
"""
As we move to more ambgious topics, I would try to add in more comments on the approaches and results.
"""

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd 
import math
import random
import itertools
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.regressionplots import *
from sklearn import datasets, linear_model
from sklearn.decomposition import PCA 
from sklearn.cross_decomposition import PLSRegression, PLSSVD
from sklearn.preprocessing import StandardScaler, scale 
from sklearn import linear_model 
from sklearn.model_selection import cross_val_predict 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV


In [ ]:
# read the data and take a look at the data
Hitters = pd.read_csv('data/Hitters.csv', header=0, na_values='NA')
print(list(Hitters)) # get the header of this data
print(Hitters.shape) # get the dimension of this 
Hitters.head() # pull a sample of this dataset

In [ ]:
# clean the data a bit to remove NAs
print(np.sum(pd.isnull(Hitters['Salary']))) # number of NAs in Salary column'
print(Hitters['Salary'].isnull().sum())

Hitters = Hitters.dropna().reset_index(drop=True) # drop the observation with NA values and reindex the obs from 0
print(Hitters.shape)
print(Hitters['Salary'].isnull().sum())

In [ ]:
# do some feature engineering to prepare for the model training
y = Hitters.Salary  

"""
take care of the features 
1. change category into dummy variables 
2. Choose (n-1) dummy variable into the feature set: n is the unique values of each categorical variable.
"""

dummies = pd.get_dummies(Hitters[['League', 'Division', 'NewLeague']])
print(dummies.head())
X_prep = Hitters.drop (['Salary', 'League', 'Division', 'NewLeague'], axis = 1).astype('float64')
X = pd.concat([X_prep,  dummies[['League_A', 'Division_E', 'NewLeague_A']]], axis=1)

In [ ]:
# let us get it ready for best subset selection 
"""
Since in Python there is no well-defined function for best subset selection, 
we will need to define some functions ourselves.
1. Define a function to run on a subset of feature and extract RSS
2. Select the best model (models) for a fix number of features
"""
def getRSS(y, X, feature_list):
    model = sm.OLS(y, X[list(feature_list)]).fit()
    RSS = ((model.predict(X[list(feature_list)]) - y) ** 2).sum()
    return {'Model':model, "RSS":RSS}

def bestModel(y, X, K):
    results = []
    for c in itertools.combinations(X.columns, K):
        results.append(getRSS(y, X, c))     
    model_all =  pd.DataFrame(results)
    
    best_model = model_all.loc[model_all["RSS"].idxmin()] ## this could be modified to have the top several models
    return best_model

In [ ]:
# execute the best model selection 
models = pd.DataFrame(columns=["RSS", "Model"])
for i in range(1,(max_feature+1)):  # for illustration purpuse, I just run for 1 - max_fearure features 
    models.loc[i] = bestModel(y, X, i)
    
print(models.loc[2, 'Model'].summary())


In [ ]:
# this show an example to plot the RSS of best models with different number of parameters
plt.figure()
plt.plot(models["RSS"])
plt.xlabel('# features')
plt.ylabel('RSS')
plt.show()

In [ ]:
# find the adjust R^2, use dir() to identify all available attributes
rsquared_adj = models.apply(lambda row: row[1].rsquared_adj, axis=1) 

In [ ]:
"""
The following graph shows the adj R^2 is still increasing, 
in this case, it is a good idea trying models with more features. 
"""
plt.figure()
plt.plot(rsquared_adj)
plt.xlabel('# features')
plt.ylabel('Adjust R^2')
plt.show()

## 6.5.2 Forward and Backward Stepwise Selection

### Forward Stepwise Selection

In [ ]:
# we can use the previous user defined function 'def getRSS(y, X, feature_list)' 
# to add 1 feature at a time (start from 0 feature) for forward stepwise selection
# or delete 1 feature at a time(start from all the features) for backward stepwise selection. 
def forward_select(y, X, feature_list):
    remaining_predictors = [p for p in X.columns if p not in feature_list]
    results = []
    for p in remaining_predictors:
        results.append(getRSS(y, X, feature_list+[p]))

    models = pd.DataFrame(results)
    best_model = models.loc[models['RSS'].idxmin()]
    return best_model  

In [ ]:
models2 = pd.DataFrame(columns=["RSS", "Model"])
feature_list = []
for i in range(1,len(X.columns)+1):
    models2.loc[i] = forward_select(y, X, feature_list)
    feature_list = models2.loc[i]["Model"].model.exog_names

In [ ]:
# we can compare the results of best subset selection and the forward selection
print('Best max_feature variable from best subset selection on tranining')
print(models.loc[max_feature, 'Model'].params)
print('\n---------------------------------------------')
print('Best max_feature variable from forward selection on tranining')
print(models2.loc[max_feature, 'Model'].params)

### Backward Stepwise Selection

In [ ]:
def backward_select(y, X, feature_list):
    results = []
    for combo in itertools.combinations(feature_list, len(feature_list)-1):
        results.append(getRSS(y, X, combo))

    models = pd.DataFrame(results)
    best_model = models.loc[models['RSS'].idxmin()]
    return best_model  

In [ ]:
# the backward selection starts from all the variables of features
models3 = pd.DataFrame(columns=["RSS", "Model"], index = range(1,len(X.columns)))
feature_list = X.columns

while(len(feature_list) > 1):
    models3.loc[len(feature_list)-1] = backward_select(y, X, feature_list)
    feature_list = models3.loc[len(feature_list)-1]["Model"].model.exog_names

print(models3.loc[max_feature, "Model"].params)

## 6.5.3 Choosing Among Models Using the Validation Set Approach and Cross-Validation
In previous sections, we defined the 'best' model based on some statistics (R^2, adj R^2, AIC, BIC, etc) of the training dataset. This may cause 'overfitting' problemm which means the best model on training data can not generalize well to new data. In this section, validation approach will be discussed.


In [ ]:
# randomly split the data into traning dataset and validation dateset
np.random.seed(seed = 21)
train_index = np.random.choice([True, False], size = len(y), replace = True, p = [0.7, 0.3]) 
# random select ~70% of data into traning sample
# the rest of the samples will be in testing set.
test_index = np.invert(train_index)
X_train= X[train_index]
y_train = y[train_index]
X_test = X[test_index]
y_test = y[test_index]

In [ ]:
""" 
We can recyle the old functions. Modification is needed to compute the RSS for the testing data. 
So we need to add both train and test into the function input (Implement)
-OR-: we can wrap the train and test split step into the function(Not Implemented)
"""
def getRSS_validation(y_train, X_train, y_test, X_test,  feature_list):
    model = sm.OLS(y_train, X_train[list(feature_list)]).fit()
    RSS = ((model.predict(X_test[list(feature_list)]) - y_test) ** 2).sum()
    return {'Model':model, "RSS":RSS}

def bestModel_validation(y_train, X_train, y_test, X_test, K):
    results = []
    for c in itertools.combinations(X_train.columns, K):
        results.append(getRSS_validation(y_train, X_train, y_test, X_test, c))     
    model_all =  pd.DataFrame(results)
    
    best_model = model_all.loc[model_all["RSS"].idxmin()] ## this could be modified to have the top several models
    return best_model


def forward_select_validation(y_train, X_train, y_test, X_test,  feature_list):
    remaining_predictors = [p for p in X_train.columns if p not in feature_list]
    results = []
    for p in remaining_predictors:
        results.append(getRSS_validation(y_train, X_train, y_test, X_test, feature_list+[p]))

    models = pd.DataFrame(results)
    best_model = models.loc[models['RSS'].idxmin()]
    return best_model

def backward_select_validation(y_train, X_train, y_test, X_test,  feature_list):
    results = []
    for combo in itertools.combinations(feature_list, len(feature_list)-1):
        results.append(getRSS_validation(y_train, X_train, y_test, X_test,  combo))

    models = pd.DataFrame(results)
    best_model = models.loc[models['RSS'].idxmin()]
    return best_model  

In [ ]:
models_validation = pd.DataFrame(columns=["RSS", "Model"])
for i in range(1,(max_feature+1)):  # for illustration purpuse, I just run for 1 - max_fearure features 
    models_validation.loc[i] = bestModel_validation(y_train, X_train, y_test, X_test, i) 
    
    
"""
change the function to  forward_select_validation (.) or backward_select_validation(.) 
for forward selection or backward selection
""" 
    
models2_forward = pd.DataFrame(columns=["RSS", "Model"])
feature_list = []
for i in range(1,len(X.columns)+1):
    models2_forward.loc[i] = forward_select_validation(y_train, X_train, y_test, X_test,  feature_list)
    feature_list = models2_forward.loc[i]["Model"].model.exog_names    


In [ ]:
print('Best max_feature variable from best subset selection on tranining')
print(models.loc[max_feature, 'Model'].params)
print('\n---------------------------------------------')
print('Best max_feature variable from forward selection on tranining')
print(models2.loc[max_feature, 'Model'].params)
print('\n---------------------------------------------')
print('Best max_feature variable from backward selection on tranining')
print(models3.loc[max_feature, 'Model'].params)
print('\n---------------------------------------------')
print('Best max_feature variable from best subset selection on traning and validation split')
print(models_validation.loc[max_feature, 'Model'].params)

In [ ]:
# this show an example to plot the RSS of best models with different number of parameters for best subset with validation
plt.figure()
plt.plot(models_validation["RSS"])
plt.xlabel('# features')
plt.ylabel('RSS')
plt.show()

In [ ]:
# this show an example to plot the RSS of best models with different number of parameters for forward selection with validation
plt.figure()
plt.plot(models2_forward["RSS"])
plt.xlabel('# features')
plt.ylabel('RSS')
plt.show()
"""
Note:from above graph, that 6 variables model gives us the best RSS under forward selection. 
To learn the final model, it is also recommendated to re-train the model on entire data (train + validation). 
"""

### Cross-Validation

In [ ]:
"""
This approach is similar to the previous validation idea. 
The difference is that we break the entire dataset into K different folds. 
Each run, the model will be trained on all the data from K-1 folds and tested on the remaining fold.
Advantages: 
1. There will be multiple metrics out of testing results => distribution of testing RSS, etc
2. The size of traning dataset is much closer to the size of original dateset. This will remove some biases caused by
the size difference.
"""

k = 10
np.random.seed(seed = 21)
train_index = np.random.choice(k, size = len(y), replace = True)  # Randomly assign each observations into folds
cv_errors = pd.DataFrame(columns=range(1,k+1), index=range(1,len(X.columns) + 1))

In [ ]:
# train and test on each fold
models_cv = pd.DataFrame(columns=["RSS", "Model"])
for j in range(1,k+1):
    feature_list = []
    for i in range(1,len(X.columns)+1):
        models_cv.loc[i] = forward_select_validation(y[train_index!= (j-1)], X[train_index != (j-1)], 
                                                     y[train_index == (j-1)],X[train_index == (j-1)], 
                                                     feature_list)
        
        cv_errors[j][i] = models_cv.loc[i]["RSS"]
        feature_list = models_cv.loc[i]["Model"].model.exog_names

In [ ]:
# similar to above, let us plot the performance of each model 
cv_errors_mean = cv_errors.mean(axis = 1)
plt.figure()
plt.plot(cv_errors_mean)
plt.xlabel('# features')
plt.ylabel('RSS')
plt.show()

In [ ]:
"""
From the above plot, we can see that the model with 5 variables yielded the smallest RSS.
We can take a closer look at that model summary. 
We can also see that the model performance for variables 4 - 12 are similar.
"""
print(models_cv.loc[5, "Model"].summary())

## 6.5.2 Ridge Regression and the Lasso

In [ ]:
"""
Ridge and Lasso are the two regularization methods to deal with overfitting. 
We do not to run multiple backward/forward selection. 
In Ridge and Lasso , we would need to run the regularization strength.
"""

# similar to before, we need to prepare the features(indepedent) variables and response(depedent) varisble.
Hitters = pd.read_csv('data/Hitters.csv', header=0, na_values='NA')
Hitters = Hitters.dropna().reset_index(drop=True) # drop the observation with NA values and reindex the obs from 0
dummies = pd.get_dummies(Hitters[['League', 'Division', 'NewLeague']])

y = Hitters.Salary  # the response variable 
X_prep = Hitters.drop (['Salary', 'League', 'Division', 'NewLeague'], axis = 1).astype('float64')
X = pd.concat([X_prep,  dummies[['League_A', 'Division_E', 'NewLeague_A']]], axis=1)

### Ridge Regression

In [ ]:
# next, we will generate a few canadidates regularization strength(in sklearn, the keyword is alphas) 
# for our Ridge regression. In R, alpha is a switch for Ridge and Lasso methods.
alphas = 10**np.linspace(10,-2,100)
print(alphas)

In [ ]:
# associated with each value of alpha is a vector of ridge regression coefficients, 
# stored in a matrix that can be accessed by coeffs. In this case, it is a 19×100, 
# 19 is the dimension of the features + (intercept needs to call separately) and 100 
# is the len of the alphas. The result is a numpy series with len 100 and len(coffes[0]) is 19. 
# In this specific implementation, the default is no intercept. 
# here I used list to store them, we could also use dictionary to store them as we did before.
ridge = Ridge(fit_intercept=True, normalize=True)
coeffs = []
intercepts = []

for a in alphas:
    ridge.set_params(alpha=a)
    ridge.fit(X, y)
    coeffs.append(ridge.coef_)
    intercepts.append(ridge.intercept_)

In [ ]:
print(len(coeffs))
print(len(coeffs[0]))
print(len(intercepts))
print(intercepts[0])

In [ ]:
# as design, as alphas get bigger, magnitude of coefficients turn to be closer to zero. 
# one thing to remember is that the decay is quite smooth
ax = plt.gca()
ax.plot(alphas, coeffs)
"""
Typically, we scale the alpha in log scale.
Try to plot it without. 
"""
ax.set_xscale('log') 
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')
plt.show()

In [ ]:
# let us take one example
"""
We may notice the coefficients l2 norm is different from R output 
I tried a few different normalization methods but still did not get the exact same output 
"""
sample = 49
print(alphas[sample])
print(math.sqrt(sum(map(lambda x:x*x, coeffs[sample]))))
print(coeffs[sample])


In [ ]:
# we now split the samples into a training set and a test set in order to estimate 
# the test error of ridge regression and the lasso. 
# Python provides a built-in function to produce training and test data set.
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.66)

In [ ]:
# train the model and do prediction on test dataset
ridge = Ridge(fit_intercept=True, normalize=True, alpha=4)
ridge.fit(X_train, y_train)             # Fit a ridge regression on the training data
pred = ridge.predict(X_test)           # Use this model to predict the test data
print(pd.Series(ridge.coef_, index=X.columns)) # Print coefficients
print(mean_squared_error(y_test, pred))        # Calculate the test MSE

In [ ]:
# to select best alpha, we will use cross validation. 
# as standard, we will report test set performance as the final performance metric
ridgecv =  RidgeCV(alphas, scoring='neg_mean_squared_error', normalize = True)
ridgecv.fit(X_train, y_train)
ridgecv.alpha_

In [ ]:
# produce the mean squred error of the selected alpha on test dataset
ridge_best = Ridge(alpha=ridgecv.alpha_, normalize=True)
ridge_best.fit(X_train, y_train)
mean_squared_error(y_test, ridge_best.predict(X_test))

In [ ]:
# we can exame the values of the coefficients 
"""
If we exam the values of the coefficients, most of them are tiny, but none of them is zero.
This is a diff between ridge and lasso. Ridge shrinks the coefficients proportionally.
"""
pd.Series(ridge_best.coef_, index=X.columns)

###  The Lasso
#### We saw that ridge regression with a wise choice of λ can outperform least squares as well as the null model on the Hitters data set. We now ask whether the lasso can yield either a more accurate or a more interpretable model than ridge regression.

In [ ]:
# same precedure as before, we will use cross validation to select the best alpha
lasso= Lasso(normalize=True, max_iter=1e5) 
coeffs = []

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(X_train, y_train)
    coeffs.append(lasso.coef_)
    
ax = plt.gca()
ax.plot(alphas, coeffs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')
plt.show()

lassocv = LassoCV(alphas=None, cv=10, max_iter=1e5, normalize=True)
lassocv.fit(X_train, y_train)

lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(X_train, y_train)
mean_squared_error(y_test, lasso.predict(X_test))

In [ ]:
# some of the coefficients should reduce to exact zero
pd.Series(lasso.coef_, index=X.columns)

## 6.5.3 PCR and PLS Regression

### Principal Components Regression
Principal components regression (PCR) can be performed using the PCA function + regression afterwards. The PCA function is part of the scikit-learn module. In this section, we will continue using Hitters data, in order to predict Salary. Again, we will drop NA and deal with the categorical variables from the data set as what we did in Section 6.5. 

In [ ]:
Hitters = pd.read_csv('data/Hitters.csv', header=0, na_values='NA')
Hitters = Hitters.dropna().reset_index(drop=True) # drop the observation with NA values and reindex the obs from 0
y = Hitters.Salary  # the response variable 
dummies = pd.get_dummies(Hitters[['League', 'Division', 'NewLeague']])
X_prep = Hitters.drop (['Salary', 'League', 'Division', 'NewLeague'], axis = 1).astype('float64')
X = pd.concat([X_prep,  dummies[['League_A', 'Division_E', 'NewLeague_A']]], axis=1)

In [ ]:
# let us do PCA on the input dataset. 
# since the units of the variables are different, it is always recommended to scale the variables.
pca = PCA()
X_pca = pca.fit_transform(scale(X))
regr = linear_model.LinearRegression()

In [ ]:
"""
Here comes a problem, how to choose the number of PCs. We still use cross-validation. 
We compute MSE for validation set at different number of PCs, 
and choose the one with lowest validation MSE as the optimal number for PCA dimension reduction. 

[NOT fully covered here, covered in Chapter 12] We may decide on the number of PCS by the explained variance ratio.
"""

def pcr(X,y,pc):
    ''' Principal Component Regression in Python'''
    ''' Step 1: PCA on input data'''

    # Define the PCA object
    pca = PCA()

    # Preprocessing (1): first derivative
    X_pca = pca.fit_transform(scale(X))[:,:pc]

    ''' Step 2: regression on selected principal components'''

    # Create linear regression object
    regr = linear_model.LinearRegression()
    
    # Fit
    regr.fit(X_pca, y)

    # Calibration
    y_train = regr.predict(X_pca)

    # Cross-validation
    y_cv = cross_val_predict(regr, X_pca, y, cv=20)

    # Calculate scores for training and cross-validation
    score_train = r2_score(y, y_train)
    score_cv = r2_score(y, y_cv)

    # Calculate mean square error for training and cross validation
    mse_train = mean_squared_error(y, y_train)
    mse_cv = mean_squared_error(y, y_cv)

    return(y_cv, score_train, score_cv, mse_train, mse_cv)


In [ ]:
mse_train = []
mse_cv = []

# calculate MSE using CV for the 19 principle components, adding one component at the time.
for i in np.arange(1, 20):
    results =  pcr(X,y,i)
    mse_train.append(results[3])
    mse_cv.append(results[4])
    
# plot results    
plt.plot(np.arange(1, 20), mse_cv, '-v', label = 'Validation_MSE')
plt.plot(np.arange(1, 20), mse_train, '-v', label = 'Train_MSE')
plt.xlabel('Number of principal components in regression')
plt.ylabel('MSE')
plt.title('Salary')
plt.xlim(left=-1)
plt.legend()
plt.show()

"""
From the above picture, we can observe three things 
1)PC = 6, 16, 17, 18, 19 give us the small validation MSE; 
2) for all the PC dimensions,  validation MSE is higher than training MSE, is this normal? 
3) the training MSE keeps decreasing as PC number goes up, is this as expected?


From the plot we also see that the cross-validation error is roughly the same when 
only one component is included in the model. 
This suggests that a model that uses just a small number of components might suffice. 
In the book, the authors used train/test to select the best dimension. 
I will skip that part since most of those were already covered in the previous sections.
"""

In [ ]:
# let us move forwward with 6 PCs
pcs = 6
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.66)

pca2 = PCA()
# scale the data
X_reduced_train = pca2.fit_transform(scale(X_train))
X_reduced_test = pca2.transform(scale(X_test))[:,:pcs]
# train regression model on training data 
regr = linear_model.LinearRegression()
regr.fit(X_reduced_train[:,:6], y_train)
# prediction with test data
pred = regr.predict(X_reduced_test)
mean_squared_error(y_test, pred)

In [ ]:
"""
We'll do a little math to get the amount of variance explained by adding each consecutive principal component. 
We can think of this as the amount of information(variance) about the data(X) or the response that is 
captured using $M$ principal components. 
For example, setting $M = 1$ only captures 38.31% of all the variance, or information, in the data.
"""
np.cumsum(pca.explained_variance_ratio_) * 100

### Partial Least Squares
#### Scikit-learn has function PLSRegression for partial least squares regression. But, we still need to write a few line of codes to do the cross validation. The logic is same as before. 

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.66)
pls = PLSRegression(n_components=2)
pls.fit(scale(X_train), y_train)

mean_squared_error(y_test, pls.predict(scale(X_test)))

In [ ]:
# End of Chapter 6